# Builder (Polyglot)

In [ ]:
#!import ../../lib/fsharp/Testing.dib

In [ ]:
#r @"../../../../../../../.nuget/packages/fsharp.control.asyncseq/3.2.1/lib/netstandard2.1/FSharp.Control.AsyncSeq.dll"
#r @"../../../../../../../.nuget/packages/system.reactive/6.0.1-preview.1/lib/net6.0/System.Reactive.dll"
#r @"../../../../../../../.nuget/packages/system.reactive.linq/6.0.1-preview.1/lib/netstandard2.0/System.Reactive.Linq.dll"
#r @"../../../../../../../.nuget/packages/argu/6.1.1/lib/netstandard2.0/Argu.dll"
#r @"../../../../../../../.nuget/packages/system.commandline/2.0.0-beta4.22272.1/lib/net6.0/System.CommandLine.dll"

In [ ]:
#!import ../../lib/fsharp/Common.fs
#!import ../../lib/fsharp/CommonFSharp.fs
#!import ../../lib/fsharp/Async.fs
#!import ../../lib/fsharp/AsyncSeq.fs
#!import ../../lib/fsharp/Networking.fs
#!import ../../lib/fsharp/Runtime.fs
#!import ../../lib/fsharp/FileSystem.fs

In [ ]:
open Common
open FileSystem.Operators

## buildProject

In [ ]:
let inline buildProject runtime path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let fileDir = fullPath |> System.IO.Path.GetDirectoryName
    let extension = fullPath |> System.IO.Path.GetExtension

    let getLocals () = $"fullPath: {fullPath} / {getLocals ()}"
    trace Debug (fun () -> "buildProject") getLocals

    match extension with
    | ".fsproj" -> ()
    | _ -> failwith "Invalid project file"

    let runtimes =
        runtime
        |> Option.map List.singleton
        |> Option.defaultValue [ "linux-x64"; "win-x64" ]

    return!
        runtimes
        |> List.map (fun runtime -> async {
            let! exitCode, _result =
                Runtime.executeWithOptionsAsync
                    {
                        Command = $@"dotnet publish ""{path}"" --configuration Release --output ../dist --runtime {runtime}"
                        CancellationToken = None
                        OnLine = None
                        WorkingDirectory = Some fileDir
                    }

            return exitCode
        })
        |> Async.Sequential
        |> Async.map Array.sum
}

## persistCodeProject

In [ ]:
let inline persistCodeProject packages modules path name code = async {
    let getLocals () = $"packages: {packages} / modules: {modules} / path: {path} / name: {name} / code.Length: {code |> String.length} / {getLocals ()}"
    trace Debug (fun () -> "persistCodeProject") getLocals

    let targetPath = path </> "target"
    System.IO.Directory.CreateDirectory targetPath |> ignore

    let filePath = targetPath </> $"{name}.fs" |> System.IO.Path.GetFullPath
    do! code |> FileSystem.writeAllTextExists filePath

    let repositoryRoot = FileSystem.getSourceDirectory () |> FileSystem.findParent ".paket" false

    let modulesCode =
        modules
        |> List.map (fun path -> $"""<Compile Include="{repositoryRoot </> path}" />""")
        |> String.concat "\n        "

    let fsprojPath = targetPath </> $"{name}.fsproj"
    let fsprojCode = $"""<Project Sdk="Microsoft.NET.Sdk">
    <PropertyGroup>
        <TargetFramework>net8.0</TargetFramework>
        <LangVersion>preview</LangVersion>
        <RollForward>Major</RollForward>
        <TargetLatestRuntimePatch>true</TargetLatestRuntimePatch>
        <PublishAot>false</PublishAot>
        <PublishTrimmed>false</PublishTrimmed>
        <PublishSingleFile>true</PublishSingleFile>
        <SelfContained>true</SelfContained>
        <Version>0.0.1-alpha.1</Version>
        <OutputType>Exe</OutputType>
    </PropertyGroup>

    <ItemGroup>
        {modulesCode}
        <Compile Include="{filePath}" />
    </ItemGroup>

    <Import Project="{repositoryRoot}/.paket/Paket.Restore.targets" />
</Project>
"""
    do! fsprojCode |> FileSystem.writeAllTextExists fsprojPath

    let paketReferencesPath = targetPath </> "paket.references"
    let paketReferencesCode =
        "FSharp.Core" :: packages
        |> String.concat "\n"
    do! paketReferencesCode |> FileSystem.writeAllTextExists paketReferencesPath

    return fsprojPath
}

## buildCode

In [ ]:
let inline buildCode runtime packages modules path name code = async {
    let! fsprojPath = persistCodeProject packages modules path name code
    return! fsprojPath |> buildProject runtime
}

In [ ]:
//// test

let tempFolder = FileSystem.getSourceDirectory () </> "target/test"

"1 + 1"
|> buildCode None [] [] tempFolder "test"
|> Async.runWithTimeout 60000
|> _equal (Some 0)

00:00:00 #1 [Debug] persistCodeProject / packages: [] / modules: [] / path: C:\home\git\polyglot\apps\builder\target/test / name: test / code.Length: 5
00:00:00 #2 [Debug] buildProject / fullPath: C:\home\git\polyglot\apps\builder\target\test\target\test.fsproj
00:00:00 #3 [Debug] executeAsync / options: { Command =
   "dotnet publish "C:\home\git\polyglot\apps\builder\target/test\target\test.fsproj" --configuration Release --output ../dist --runtime linux-x64"
  WorkingDirectory = Some "C:\home\git\polyglot\apps\builder\target\test\target"
  CancellationToken = None
  OnLine = None }
00:00:00 #4 [Verbose] > MSBuild version 17.8.0-preview-23418-03+0125fc9fb for .NET
00:00:00 #5 [Verbose] >   Determining projects to restore...
00:00:01 #6 [Verbose] >   Restored C:\home\git\polyglot\apps\builder\target\test\target\test.fsproj (in 351 ms).
00:00:01 #7 [Verbose] > C:\Program Files\dotnet\sdk\8.0.100-rc.1.23463.5\Sdks\Microsoft.NET.Sdk\targets\Microsoft.NET.RuntimeIdentifierInference.target

In [ ]:
//// test

let tempFolder = FileSystem.getSourceDirectory () </> "target/test"

"1 + a"
|> buildCode None [] [] tempFolder "test"
|> Async.runWithTimeout 60000
|> _equal (Some 2)

00:00:12 #23 [Debug] persistCodeProject / packages: [] / modules: [] / path: C:\home\git\polyglot\apps\builder\target/test / name: test / code.Length: 5
00:00:12 #24 [Debug] buildProject / fullPath: C:\home\git\polyglot\apps\builder\target\test\target\test.fsproj
00:00:12 #25 [Debug] executeAsync / options: { Command =
   "dotnet publish "C:\home\git\polyglot\apps\builder\target/test\target\test.fsproj" --configuration Release --output ../dist --runtime linux-x64"
  WorkingDirectory = Some "C:\home\git\polyglot\apps\builder\target\test\target"
  CancellationToken = None
  OnLine = None }
00:00:12 #26 [Verbose] > MSBuild version 17.8.0-preview-23418-03+0125fc9fb for .NET
00:00:12 #27 [Verbose] >   Determining projects to restore...
00:00:13 #28 [Verbose] >   Restored C:\home\git\polyglot\apps\builder\target\test\target\test.fsproj (in 325 ms).
00:00:13 #29 [Verbose] > C:\Program Files\dotnet\sdk\8.0.100-rc.1.23463.5\Sdks\Microsoft.NET.Sdk\targets\Microsoft.NET.RuntimeIdentifierInference

## buildFile

In [ ]:
let inline buildFile runtime packages modules path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let dir = fullPath |> System.IO.Path.GetDirectoryName
    let fileName = fullPath |> System.IO.Path.GetFileNameWithoutExtension
    let! code = fullPath |> FileSystem.readAllTextAsync

    let code = System.Text.RegularExpressions.Regex.Replace (
        code,
        @"( *)(let\s+main\s+.*?\s*=)",
        fun m -> m.Groups.[1].Value + "[<EntryPoint>]\n" + m.Groups.[1].Value + m.Groups.[2].Value
    )

    let code =
        if code |> String.endsWith "\n()\n"
        then code |> String.substring 0 ((code |> String.length) - 4)
        else code

    return! code |> buildCode runtime packages modules dir fileName
}

## Arguments

In [ ]:
[<RequireQualifiedAccess>]
type Arguments =
    | [<Argu.ArguAttributes.MainCommand; Argu.ArguAttributes.ExactlyOnce>] Path of path : string
    | [<Argu.ArguAttributes.Unique>] Packages of packages : string list
    | [<Argu.ArguAttributes.Unique>] Modules of modules : string list
    | [<Argu.ArguAttributes.Unique>] Runtime of runtime : string

    interface Argu.IArgParserTemplate with
        member s.Usage =
            match s with
            | Path _ -> nameof Path
            | Packages _ -> nameof Packages
            | Modules _ -> nameof Modules
            | Runtime _ -> nameof Runtime

## main

In [ ]:
let main args =
    let argsMap = args |> Runtime.parseArgsMap<Arguments>

    let path =
        match argsMap.[nameof Arguments.Path] with
        | [ Arguments.Path path ] -> Some path
        | _ -> None
        |> Option.get

    let packages =
        match argsMap |> Map.tryFind (nameof Arguments.Packages) with
        | Some [ Arguments.Packages packages ] -> packages
        | _ -> []

    let modules =
        match argsMap |> Map.tryFind (nameof Arguments.Modules) with
        | Some [ Arguments.Modules modules ] -> modules
        | _ -> []

    let runtime =
        match argsMap |> Map.tryFind (nameof Arguments.Runtime) with
        | Some [ Arguments.Runtime runtime ] -> Some runtime
        | _ -> None

    path
    |> buildFile runtime packages modules
    |> Async.runWithTimeout 60000
    |> function
        | Some exitCode -> exitCode
        | None -> 1

In [ ]:
//// test

let args =
    System.Environment.GetEnvironmentVariable "ARGS"
    |> Runtime.splitArgs
    |> Seq.toArray

match args with
| [||] -> 0
| args -> if main args = 0 then 0 else failwith "main failed"

0

00:00:19 #39 [Debug] persistCodeProject / packages: [Argu; FSharp.Control.AsyncSeq; System.CommandLine; ... ] / modules: [lib/fsharp/Common.fs; lib/fsharp/CommonFSharp.fs; lib/fsharp/Async.fs; ... ] / path: C:\home\git\polyglot\apps\builder / name: Builder / code.Length: 6181
00:00:19 #40 [Debug] buildProject / fullPath: C:\home\git\polyglot\apps\builder\target\Builder.fsproj
00:00:19 #41 [Debug] executeAsync / options: { Command =
   "dotnet publish "C:\home\git\polyglot\apps\builder\target\Builder.fsproj" --configuration Release --output ../dist --runtime linux-x64"
  WorkingDirectory = Some "C:\home\git\polyglot\apps\builder\target"
  CancellationToken = None
  OnLine = None }
00:00:19 #42 [Verbose] > MSBuild version 17.8.0-preview-23418-03+0125fc9fb for .NET
00:00:20 #43 [Verbose] >   Determining projects to restore...
00:00:21 #44 [Verbose] >   Restored C:\home\git\polyglot\apps\builder\target\Builder.fsproj (in 383 ms).
00:00:21 #45 [Verbose] > C:\Program Files\dotnet\sdk\8.0.100